In [1]:
from diffusers import StableDiffusionUpscalePipeline
from PIL import Image
import torch
import imageio as io
import numpy as np
import os
import glob
import natsort
from tqdm import tqdm

/home/ubuntu/Environment/doc_manglam/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-30 06:53:27.378072: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-30 06:53:27.430095: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 06:53:28.227898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Coul

ModuleNotFoundError: No module named 'natsort'

In [2]:
# This function will squarify images as per the larger dimension and make it a square image with pad values 
def squarify_image(image):

    h,w = image.shape[0], image.shape[1]

    border = int(h//2)
    image_square = np.zeros((w,w,3))
    # print(image.shape, image_square.shape)
    # print(image_square[border:w-border, :, :].shape)

    image_square[w//2 - border:w//2 + h - border, :, :] = image
    return image_square

In [3]:
def unpad_image(image, size):
    h, w = size[0], size[1]
    image = image.resize((w,w))
    npimg = np.array(image)
    
    border = int(h//2)

    image_unpad = npimg[w//2 - border:w//2 + h - border, :, :]
    return Image.fromarray(image_unpad)


In [4]:
# load model and scheduler
device = torch.device('cuda')
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16)
pipeline = pipeline.to(device)
pipeline.enable_attention_slicing()
pipeline.enable_xformers_memory_efficient_attention()

unet/diffusion_pytorch_model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|██████████| 6/6 [00:25<00:00,  4.28s/it]
/home/ubuntu/Environment/doc_manglam/lib/python3.8/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_upscale.py:123: FutureWarning: The configuration file of the vae does not contain `scaling_factor` or it is set to 0.18215, which seems highly unlikely. If your checkpoint is a fine-tuned version of `stabilityai/stable-diffusion-x4-upscaler` you should change 'scaling_factor' to 0.08333 Please make sure to update the config accordingly, as not doing so might lead to incorrect results in future versions. If you have downloaded this checkpo

In [11]:
source_image_path = 'Data/svp.jpg'

In [ ]:
source_image = io.imread(source_image_path)
w, h = source_image.shape[:-1]

input_image = Image.fromarray(source_image.astype('uint8'), 'RGB').resize((256,256))


prompt = ''
upscaled_image = pipeline(prompt=prompt, image=input_image).images[0]
upscaled_image = upscaled_image.resize((h, w))

upscaled_image.save(source_image_path.replace('.jpg', '_SR.jpg'))

/tmp/ipykernel_202077/4294010167.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  source_image = io.imread(source_image_path)
  0%|          | 0/75 [00:00<?, ?it/s]